In [1]:
!mkdir data/ 
!wget https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar -O data/ILSVRC2012_img_val.tar
!tar -xf data/ILSVRC2012_img_val.tar -C data/
!wget https://raw.githubusercontent.com/Evolving-AI-Lab/ppgn/master/misc/map_clsloc.txt
!wget https://raw.githubusercontent.com/yqihao/PTMValidations/main/imagenet_classes.txt
!wget https://raw.githubusercontent.com/yqihao/PTMValidations/main/ILSVRC2012_validation_ground_truth.txt

--2022-05-03 15:51:39--  https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar
Resolving image-net.org (image-net.org)... 171.64.68.16
Connecting to image-net.org (image-net.org)|171.64.68.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6744924160 (6.3G) [application/x-tar]
Saving to: ‘data/ILSVRC2012_img_val.tar’

data/ILSVRC2012_img 100%[===================>]   6.28G  30.6MB/s    in 10m 15s 

2022-05-03 16:01:54 (10.5 MB/s) - ‘data/ILSVRC2012_img_val.tar’ saved [6744924160/6744924160]

--2022-05-03 16:02:32--  https://raw.githubusercontent.com/Evolving-AI-Lab/ppgn/master/misc/map_clsloc.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24366 (24K) [text/plain]
Saving to: ‘map_clsloc.txt’

map_clsloc.txt      100%[====

In [2]:
from torchvision import models
import torch
from PIL import Image

In [3]:
target_model = models.efficientnet_b5(pretrained=True)
target_model.eval()

Downloading: "https://download.pytorch.org/models/efficientnet_b5_lukemelas-b6417697.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b5_lukemelas-b6417697.pth


  0%|          | 0.00/117M [00:00<?, ?B/s]

EfficientNet(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(48, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): ConvNormActivation(
            (0): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
            (1): BatchNorm2d(48, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): ConvNormActivatio

In [4]:
from torchvision import transforms
transform = transforms.Compose([            #[1]
 transforms.Resize(256),                    #[2]
 transforms.CenterCrop(224),                #[3]
 transforms.ToTensor(),                     #[4]
 transforms.Normalize(                      #[5]
 mean=[0.485, 0.456, 0.406],                #[6]
 std=[0.229, 0.224, 0.225]                  #[7]
 )])

In [5]:
with open('imagenet_classes.txt') as f:
  classes = [line.strip() for line in f.readlines()]

mapping_file = open('map_clsloc.txt',mode='r')

validation_file = open('ILSVRC2012_validation_ground_truth.txt',mode='r')
validation_list = validation_file.readlines()
mapping_list_from = mapping_file.readlines()
mapping_list_to = []
classes_mapping_list = []
classes_mapping = {}
for i in range(len(mapping_list_from)):
  classes_mapping[mapping_list_from[i].split(" ")[2].replace('\n',"")] = mapping_list_from[i].split(" ")[1]
for i in range(len(classes)):
  classes_mapping_list.append(classes_mapping[classes[i].split(", ")[1]])

for i in range(len(validation_list)):
  validation_list[i] = validation_list[i].split('\n')[0]

In [6]:
correct = 0
top_5 = 0

it = 50000

for i in range(it):
  img_path = f'data/ILSVRC2012_val_{(i+1):08d}.JPEG'
  # print(img_path)
  # img = Image.open("dog.jpg")
  img = Image.open(img_path).convert('RGB')
  img_t = transform(img)
  batch_t = torch.unsqueeze(img_t, 0)

  out = target_model(batch_t)

  _, index = torch.topk(input=out,k=5)

  # print(index[0].tolist())

  if classes_mapping_list[index[0][0]] == validation_list[i]:
    correct = correct + 1
    top_5 = top_5 + 1
  else:
    for ii in range(1,5):
      if classes_mapping_list[index[0][ii]] == validation_list[i]:
        top_5 = top_5 + 1
        break
  # print(correct, top_5)
  if i!=0 and i%500==0:
    print(str(int(i/500))+'%, Top 1:' + str(correct/i) + ' Top 5:' + str(top_5/i))

accuracy = correct/it*100
top_5_accuracy = top_5/it*100
print('Top 1 Accuracy:'+str(accuracy)+'%')
print('Top 5 Accuracy:'+str(top_5_accuracy)+'%')

1%, Top 1:0.722 Top 5:0.918
2%, Top 1:0.718 Top 5:0.906
3%, Top 1:0.7306666666666667 Top 5:0.908
4%, Top 1:0.7145 Top 5:0.906
5%, Top 1:0.7252 Top 5:0.9104
6%, Top 1:0.7223333333333334 Top 5:0.9093333333333333
7%, Top 1:0.72 Top 5:0.9111428571428571
8%, Top 1:0.722 Top 5:0.91025
9%, Top 1:0.7222222222222222 Top 5:0.9093333333333333
10%, Top 1:0.7242 Top 5:0.908
11%, Top 1:0.726 Top 5:0.9094545454545454
12%, Top 1:0.7276666666666667 Top 5:0.911
13%, Top 1:0.7286153846153847 Top 5:0.9110769230769231
14%, Top 1:0.7274285714285714 Top 5:0.9094285714285715
15%, Top 1:0.7293333333333333 Top 5:0.9104
16%, Top 1:0.730125 Top 5:0.90975
17%, Top 1:0.7303529411764705 Top 5:0.9097647058823529
18%, Top 1:0.7287777777777777 Top 5:0.9088888888888889
19%, Top 1:0.7308421052631578 Top 5:0.9092631578947369
20%, Top 1:0.7304 Top 5:0.9079
21%, Top 1:0.7310476190476191 Top 5:0.9083809523809524
22%, Top 1:0.7307272727272728 Top 5:0.9080909090909091
23%, Top 1:0.7302608695652174 Top 5:0.9077391304347826
24%,